In [1]:
# TODO: Adapt this for two-gene perts such that we are only testing the two-gene perts

In [3]:
import os.path

import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from tqdm import tqdm
import scanpy as sc
from src.utils.spectra.dataset import SpectraDataset
from src.utils.spectra.perturb import Spectra
from gears import PertData
import pickle as pkl

In [4]:
class PerturbGraphData(SpectraDataset):
    def parse(self, pert_data):
        if isinstance(pert_data, PertData):
            self.adata = pert_data.adata
        else:
            self.adata = pert_data
        self.control_expression = self.adata[self.adata.obs['condition'] == 'ctrl'].X.toarray().mean(axis=0)
        return [p for p in self.adata.obs['condition'].unique() if p != 'ctrl']

    def get_mean_logfold_change(self, perturbation):
        perturbation_expression = self.adata[self.adata.obs['condition'] == perturbation].X.toarray().mean(axis=0)
        logfold_change = np.nan_to_num(np.log2(perturbation_expression + 1) - np.log2(self.control_expression + 1))
        return logfold_change

    def sample_to_index(self, sample):
        if not hasattr(self, 'index_to_sequence'):
            print("Generating index to sequence")
            self.index_to_sequence = {}
            for i in tqdm(range(len(self))):
                x = self.__getitem__(i)
                self.index_to_sequence['-'.join(list(x))] = i

        return self.index_to_sequence[sample]
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        perturbation = self.samples[idx]
        return self.get_mean_logfold_change(perturbation)

In [5]:
class SPECTRAPerturb(Spectra):
    def spectra_properties(self, sample_one, sample_two):
        return -np.linalg.norm(sample_one - sample_two)

    def cross_split_overlap(self, train, test):
        average_similarity = []

        for i in test:
            for j in train:
                average_similarity.append(self.spectra_properties(i, j))

        return np.mean(average_similarity)

In [6]:
adata = sc.read('../data/norman_2019_raw.h5ad')
data_type = 'norman_2'

In [7]:
nonzero_genes = (adata.X.sum(axis=0) > 5).A1
filtered_adata = adata[:, nonzero_genes]
adata.obs['condition'] = adata.obs['guide_ids'].cat.rename_categories({'': 'ctrl'})
# data_type = input(f"Pick the data type from ['norman_1', 'norman_2']: ")
# assert data_type in ['norman_1', 'norman_2']

data_type = "norman_2"

if data_type == "norman_1":
    single_gene_mask = [True if "," not in name else False for name in adata.obs['condition']]
    adata = filtered_adata[single_gene_mask, :]
else:
    adata.obs['condition'] = adata.obs['condition'].str.replace(',', '+')

genes = adata.var['gene_symbols'].to_list()
genes_and_ctrl = genes + ['ctrl']

# we remove the cells with perts that are not in the genes because we need gene expression values
# to generate an in-silico perturbation embedding
if data_type == "norman_1":
    adata = adata[adata.obs['condition'].isin(genes_and_ctrl), :]
else:
    conditions = adata.obs['condition']
    
    # need to account for the two-gene perturbations
    filtered_conditions = conditions.apply(
        lambda cond: cond in genes_and_ctrl or (
                '+' in cond and all(gene in genes_and_ctrl for gene in cond.split('+'))
        )
    )
    adata = adata[filtered_conditions, :]

In [8]:
perturb_graph_data = PerturbGraphData(adata, 'norman')

In [9]:
all_perts_orig = [p for p in adata.obs['condition'].unique() if p != 'ctrl']

all_perts_map = {i: pert for i, pert in enumerate(all_perts_orig)}

In [10]:
perts = PerturbGraphData.parse(perturb_graph_data, adata)
perts

['TSC22D1',
 'KLF1+MAP2K6',
 'CEBPE+RUNX1T1',
 'MAML2',
 'CEBPE',
 'ETS2+MAP7D1',
 'CBL+PTPN9',
 'ELMSAN1+LHX1',
 'ETS2+TGFBR2',
 'SGK1+TBX3',
 'DUSP9',
 'MAP2K6+SPI1',
 'ELMSAN1',
 'UBASH3B',
 'PTPN12+UBASH3B',
 'FOXA1',
 'FOXA1+FOXA3',
 'ETS2+IGDCC3',
 'BCORL1',
 'MEIS1',
 'GLB1L2',
 'KLF1',
 'OSR2+PTPN12',
 'IKZF3+MAP2K6',
 'BAK1',
 'FEV',
 'MAP2K3+SLC38A2',
 'ATL1',
 'CBL',
 'ETS2',
 'SET',
 'TBX3',
 'LHX1',
 'FOXA1+KLF1',
 'TBX2+TBX3',
 'SLC4A1',
 'RREB1',
 'ZNF318',
 'DUSP9+MAPK1',
 'COL2A1',
 'ZBTB25',
 'MAP4K5',
 'CEBPE+KLF1',
 'SLC6A9',
 'MIDN',
 'OSR2+UBASH3B',
 'DLX2',
 'UBASH3B+ZBTB25',
 'CBFA2T3',
 'HES7',
 'DUSP9+ETS2',
 'FOXL2+ZNF318',
 'CEBPE+SET',
 'PTPN9+UBASH3B',
 'IGDCC3+ZBTB25',
 'AHR',
 'FOXO4',
 'RHOXF2',
 'SPI1',
 'RUNX1T1',
 'S1PR2',
 'POU3F2',
 'IER5L+LYL1',
 'CNN1',
 'IGDCC3+MAPK1',
 'CELF2',
 'CEBPA+JUN',
 'MAP2K3',
 'MAP4K3',
 'SAMD1',
 'BAK1+TMSB4X',
 'CEBPE+ZC3HAV1',
 'CDKN1A',
 'UBASH3A+UBASH3B',
 'MAP2K3+MAP2K6',
 'MAPK1+PRTG',
 'PTPN1',
 'TBX2',
 'IER5

In [11]:
sp_ids = ["0.00", "0.10", "0.20", "0.30", "0.40", "0.50", "0.60", "0.70"]
replicates = 3 
all_sps = [f"{x}_{y}" for x in sp_ids for y in range(replicates)]
all_sps

['0.00_0',
 '0.00_1',
 '0.00_2',
 '0.10_0',
 '0.10_1',
 '0.10_2',
 '0.20_0',
 '0.20_1',
 '0.20_2',
 '0.30_0',
 '0.30_1',
 '0.30_2',
 '0.40_0',
 '0.40_1',
 '0.40_2',
 '0.50_0',
 '0.50_1',
 '0.50_2',
 '0.60_0',
 '0.60_1',
 '0.60_2',
 '0.70_0',
 '0.70_1',
 '0.70_2']

In [12]:
split_map = {}
for split in all_sps:
    split_id = f'SP_{split}'
    with open(f'../../data/splits/perturb/{data_type}/{data_type}_SPECTRA_splits/{split_id}/test.pkl', 'rb') as f:
        spectra_splits = pkl.load(f)
    test_perts = [all_perts_map[split] for split in spectra_splits]
    split_map[split_id] = test_perts

In [13]:
split_map

{'SP_0.00_0': ['CEBPA+ZC3HAV1',
  'CEBPE+ETS2',
  'BCL2L11+TGFBR2',
  'ETS2+IKZF3',
  'MIDN',
  'TBX3',
  'IER5L',
  'CEBPE+CNN1',
  'PTPN9',
  'PTPN12+ZBTB10',
  'BCORL1',
  'BPGM',
  'DUSP9+PRTG',
  'IGDCC3+PRTG',
  'SAMD1+UBASH3B',
  'CDKN1A+CDKN1C',
  'ETS2+MAPK1',
  'DUSP9+IGDCC3',
  'DUSP9+MAPK1',
  'DLX2+SNAI1',
  'CDKN1A+CDKN1B',
  'CEBPE+SET',
  'OSR2',
  'BAK1+TMSB4X',
  'FOXA1+FOXL2',
  'MAP2K6+SPI1',
  'UBASH3A',
  'MAP7D1',
  'PTPN13',
  'KIF18B+KIF2C',
  'FOXA3+FOXL2',
  'POU3F2',
  'PTPN12+SAMD1',
  'MAP2K6',
  'UBASH3B',
  'KIF2C',
  'HNF4A',
  'CDKN1C',
  'FOXA1',
  'CEBPA+CEBPE',
  'AHR+FEV',
  'CEBPE+RUNX1T1',
  'CLDN6',
  'CBL+UBASH3B',
  'ELMSAN1+MAP2K3',
  'CEBPB',
  'DUSP9+SNAI1'],
 'SP_0.00_1': ['MAP7D1',
  'FOXA3+FOXL2',
  'SAMD1+TGFBR2',
  'DUSP9',
  'LYL1',
  'ETS2+MAPK1',
  'ELMSAN1+MAP2K3',
  'CDKN1A',
  'PTPN9+UBASH3B',
  'JUN',
  'S1PR2+SGK1',
  'SNAI1',
  'CEBPE+ZC3HAV1',
  'FOXO4',
  'ARRDC3',
  'SAMD1',
  'CEBPA+CEBPE',
  'SNAI1+ZBTB10',
  'MAP4K3',
  

In [14]:
if data_type == "norman_2":
    filtered_split_map = {k: [v for v in vals if '+' in v] for k, vals in split_map.items()}
filtered_split_map

{'SP_0.00_0': ['CEBPA+ZC3HAV1',
  'CEBPE+ETS2',
  'BCL2L11+TGFBR2',
  'ETS2+IKZF3',
  'CEBPE+CNN1',
  'PTPN12+ZBTB10',
  'DUSP9+PRTG',
  'IGDCC3+PRTG',
  'SAMD1+UBASH3B',
  'CDKN1A+CDKN1C',
  'ETS2+MAPK1',
  'DUSP9+IGDCC3',
  'DUSP9+MAPK1',
  'DLX2+SNAI1',
  'CDKN1A+CDKN1B',
  'CEBPE+SET',
  'BAK1+TMSB4X',
  'FOXA1+FOXL2',
  'MAP2K6+SPI1',
  'KIF18B+KIF2C',
  'FOXA3+FOXL2',
  'PTPN12+SAMD1',
  'CEBPA+CEBPE',
  'AHR+FEV',
  'CEBPE+RUNX1T1',
  'CBL+UBASH3B',
  'ELMSAN1+MAP2K3',
  'DUSP9+SNAI1'],
 'SP_0.00_1': ['FOXA3+FOXL2',
  'SAMD1+TGFBR2',
  'ETS2+MAPK1',
  'ELMSAN1+MAP2K3',
  'PTPN9+UBASH3B',
  'S1PR2+SGK1',
  'CEBPE+ZC3HAV1',
  'CEBPA+CEBPE',
  'SNAI1+ZBTB10',
  'IER5L+LYL1',
  'CEBPE+KLF1',
  'FOSB+PTPN12',
  'AHR+FEV',
  'ELMSAN1+MAP2K6',
  'DLX2+SNAI1',
  'CBFA2T3+FEV',
  'PTPN12+UBASH3B',
  'RHOXF2+SET',
  'KLF1+MAP2K6',
  'PTPN12+UBASH3A',
  'SAMD1+UBASH3B',
  'HOXC13+ZC3HAV1',
  'FOXA1+FOXF1',
  'CEBPB+LYL1'],
 'SP_0.00_2': ['SGK1+TBX3',
  'CEBPA+JUN',
  'FOXA1+FOXL2',
  'CBL+

In [15]:
with open(f'../../data/splits/perturb/{data_type}/de_test/deg_pert_dict.pkl', 'rb') as f:
    deg_pert_dict = pkl.load(f)
deg_genes = list(deg_pert_dict.keys())
deg_genes

['AHR',
 'AHR+KLF1',
 'ARRDC3',
 'ATL1',
 'BAK1+KLF1',
 'BAK1+TMSB4X',
 'BCL2L11+TGFBR2',
 'BCORL1',
 'BPGM',
 'BPGM+SAMD1',
 'BPGM+ZBTB1',
 'C19orf26',
 'C19orf26+TGFBR2',
 'CBFA2T3',
 'CBL',
 'CBL+CNN1',
 'CBL+PTPN12',
 'CBL+PTPN9',
 'CBL+UBASH3B',
 'CDKN1A',
 'CDKN1B',
 'CEBPA',
 'CEBPA+KLF1',
 'CEBPB',
 'CEBPB+MAPK1',
 'CEBPB+PTPN12',
 'CEBPE',
 'CEBPE+ETS2',
 'CEBPE+KLF1',
 'CEBPE+PTPN12',
 'CEBPE+RUNX1T1',
 'CEBPE+SET',
 'CEBPE+ZC3HAV1',
 'CELF2',
 'CLDN6',
 'CNN1',
 'CNN1+ETS2',
 'CNN1+MAPK1',
 'CNN1+UBASH3B',
 'CNNM4',
 'COL1A1',
 'COL2A1',
 'CSRNP1',
 'DLX2',
 'DUSP9',
 'DUSP9+ETS2',
 'DUSP9+IGDCC3',
 'DUSP9+KLF1',
 'EGR1',
 'ELMSAN1',
 'ELMSAN1+LHX1',
 'ELMSAN1+MAP2K3',
 'ELMSAN1+MAP2K6',
 'ETS2',
 'ETS2+IGDCC3',
 'ETS2+IKZF3',
 'ETS2+MAP7D1',
 'ETS2+MAPK1',
 'ETS2+PRTG',
 'ETS2+TGFBR2',
 'FEV',
 'FOSB',
 'FOSB+OSR2',
 'FOSB+PTPN12',
 'FOSB+UBASH3B',
 'FOXA1',
 'FOXA1+KLF1',
 'FOXA3',
 'FOXA3+HOXB9',
 'FOXF1',
 'FOXL2',
 'FOXL2+POU3F2',
 'GLB1L2',
 'HK2',
 'HOXB9',
 'HOXC13',

In [16]:
# remove pert from split map if it is not in the deg_genes
split_map = {k: v for k, v in filtered_split_map.items() if any([pert in deg_genes for pert in v])}
split_map

{'SP_0.00_0': ['CEBPA+ZC3HAV1',
  'CEBPE+ETS2',
  'BCL2L11+TGFBR2',
  'ETS2+IKZF3',
  'CEBPE+CNN1',
  'PTPN12+ZBTB10',
  'DUSP9+PRTG',
  'IGDCC3+PRTG',
  'SAMD1+UBASH3B',
  'CDKN1A+CDKN1C',
  'ETS2+MAPK1',
  'DUSP9+IGDCC3',
  'DUSP9+MAPK1',
  'DLX2+SNAI1',
  'CDKN1A+CDKN1B',
  'CEBPE+SET',
  'BAK1+TMSB4X',
  'FOXA1+FOXL2',
  'MAP2K6+SPI1',
  'KIF18B+KIF2C',
  'FOXA3+FOXL2',
  'PTPN12+SAMD1',
  'CEBPA+CEBPE',
  'AHR+FEV',
  'CEBPE+RUNX1T1',
  'CBL+UBASH3B',
  'ELMSAN1+MAP2K3',
  'DUSP9+SNAI1'],
 'SP_0.00_1': ['FOXA3+FOXL2',
  'SAMD1+TGFBR2',
  'ETS2+MAPK1',
  'ELMSAN1+MAP2K3',
  'PTPN9+UBASH3B',
  'S1PR2+SGK1',
  'CEBPE+ZC3HAV1',
  'CEBPA+CEBPE',
  'SNAI1+ZBTB10',
  'IER5L+LYL1',
  'CEBPE+KLF1',
  'FOSB+PTPN12',
  'AHR+FEV',
  'ELMSAN1+MAP2K6',
  'DLX2+SNAI1',
  'CBFA2T3+FEV',
  'PTPN12+UBASH3B',
  'RHOXF2+SET',
  'KLF1+MAP2K6',
  'PTPN12+UBASH3A',
  'SAMD1+UBASH3B',
  'HOXC13+ZC3HAV1',
  'FOXA1+FOXF1',
  'CEBPB+LYL1'],
 'SP_0.00_2': ['SGK1+TBX3',
  'CEBPA+JUN',
  'FOXA1+FOXL2',
  'CBL+

In [17]:
# genes_to_keep = ['SLC4A1', 'IKZF3', 'GLB1L2', 'CEBPE', 'CEBPA', 'AHR']

In [18]:
# split_map_reduced = {k: v for k, v in split_map.items() if '0.00' in k or '0.70' in k}
# split_map_reduced

In [19]:
def filter_dict_by_list(input_dict, filter_list):
    filtered_dict = {}
    for key, values in input_dict.items():
        filtered_values = [value for value in values if value in filter_list]
        if filtered_values:
            filtered_dict[key] = filtered_values
    return filtered_dict

In [20]:
split_map_reduced = filter_dict_by_list(split_map_reduced, genes_to_keep)
split_map_reduced

NameError: name 'split_map_reduced' is not defined

In [ ]:
# save the pert_split_map dict to a pkl file to data 
with open(f'../../data/splits/perturb/{data_type}/pert_split_map.pkl', 'wb') as f:
    pkl.dump(split_map, f)

In [ ]:
def update_yaml_config(file_path, new_eval_pert):
    yaml = YAML()
    yaml.preserve_quotes = True
    yaml.indent(mapping=2, sequence=4, offset=2)

    with open(file_path, 'r') as f:
        config = yaml.load(f)

    # Update the eval_pert value
    config['data']['eval_pert'] = new_eval_pert

    # Save the updated config
    with open(file_path, 'w') as f:
        yaml.dump(config, f)

In [34]:
# TODO: open the experiment config file and generate as many as we need 
from ruamel.yaml import YAML 

base_config = '../../configs/experiment/mlp_norman_inference.yaml'
yaml = YAML()
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

with open(base_config, 'r') as f:
    config = yaml.load(f)
config

{'model_type': 'mlp', 'defaults': [{'override /model': 'mean'}, {'override /logger': 'wandb'}], 'total_genes': 2064, 'emb_dim': 2064, 'hidden_dim': 1024, 'mean_adjusted': False, 'save_dir': '${paths.data_dir}/${data.data_name}/pert_effects/${data.eval_pert}/pert_effect_pred_${data.fm}.pkl', 'train_date': '2024-11-22', 'timestamp': '22-56-31', 'data': {'data_name': 'norman_2', 'data_type': 'raw_expression', 'deg_eval': True, 'eval_pert': 'AHR+FEV', 'split': 0.0, 'replicate': 0, 'fm': 'mean'}, 'trainer': {'num_sanity_val_steps': 0, 'inference_mode': True, 'accelerator': 'gpu'}, 'ckpt_path': None, 'logger': {'wandb': {'tags': ['eval', 'norman', '${data.eval_pert}', '${data.fm}', 'split_${data.split}', 'replicate_${data.replicate}'], 'group': '${model_type}_${data.data_name}_${data.split}', 'project': 'perturbench-deg'}}}

In [35]:
import os 
if not os.path.exists(f'../../configs/experiment/deg_evals/'):
    os.makedirs(f'../../configs/experiment/deg_evals/')

id = 1
for i, (key, perts) in enumerate(split_map.items()):
    split = float(key.split('_')[1])
    replicate = int(key.split('_')[2])
    config['data']['split'] = split
    config['data']['replicate'] = replicate
    for j, pert in enumerate(perts):
        config['data']['eval_pert'] = pert
        with open(f'../../configs/experiment/deg_evals_mean/mlp_norman_inference_{id}.yaml', 'w') as f:
            yaml.dump(config, f)
        id += 1 